<div align='center'>
<img src="..\..\..\01_aux_files\img\Video_Game_Sales\1.png" height='50%' width='30%'/>
</div>




<h2>Project idea and data sources: <a href='https://www.linkedin.com/in/fabian-werkmeister/'>Fabian Werkmeister (LinkedIn)</a></h2>
<h3>Project's full material from Damian's notion: <a href='https://fabianwerkmeister.notion.site/Video-Game-Sales-8249d8cddf5c44fbb58712d950a52a91'>Click Here!</a></h3>

<h1 align='center'>FULL ETL</h1>

#### We will host our database on a local SQL Server DataWarehouse, updating and inserting data. We will refresh our data --Hopefully-- on a daily basis, and we will show the results of the Data Analytics track with Power BI.
#### ¡¡¡Even when our local server is not reacheble from outside, we won't save our credentials in the actual code, just as a good practice!!!

## Needed Libraries

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text, MetaData, Table, select

## Loading data on SRC schema, scrapping data from web

!!!<br/>
If you checked https://www.vgchartz.com/gamedb/ table, you'd probably realize that there are some records without a Last Updated date. We are not able to bring them without scanning the whole result set, because you can't filter Last Update Date value from URL.
So, we are bringing them on full loads only (running a full load everyday in a production environment would be expensive).
On Delta (incremental) refreshes, we will bring just the latest 1000 updated/added records, because we can sort results by date.<br/>
!!!

In [2]:
PMODE = "FULL"
%run "Web_Scrapping_And_Pandas/web_Scrapping_01.ipynb"

Mode: FULL
Number of videogames:  63927  -  Number of Pages:  320
Page 1 Succesfully Scrapped!
Page 2 Succesfully Scrapped!
Page 3 Succesfully Scrapped!
Page 4 Succesfully Scrapped!
Page 5 Succesfully Scrapped!
Page 6 Succesfully Scrapped!
Page 7 Succesfully Scrapped!
Page 8 Succesfully Scrapped!
Page 9 Succesfully Scrapped!
Page 10 Succesfully Scrapped!
Page 11 Succesfully Scrapped!
Page 12 Succesfully Scrapped!
Page 13 Succesfully Scrapped!
Page 14 Succesfully Scrapped!
Page 15 Succesfully Scrapped!
Page 16 Succesfully Scrapped!
Page 17 Succesfully Scrapped!
Page 18 Succesfully Scrapped!
Page 19 Succesfully Scrapped!
Page 20 Succesfully Scrapped!
Page 21 Succesfully Scrapped!
Page 22 Succesfully Scrapped!
Page 23 Succesfully Scrapped!
Page 24 Succesfully Scrapped!
Page 25 Succesfully Scrapped!
Page 26 Succesfully Scrapped!
Page 27 Succesfully Scrapped!
Page 28 Succesfully Scrapped!
Page 29 Succesfully Scrapped!
Page 30 Succesfully Scrapped!
Page 31 Succesfully Scrapped!
Page 32 Succe

In [3]:
directory = '../../../01_aux_files/results/Video_Game_Sales/Web Scrapping & Pandas/'
%run "Web_Scrapping_And_Pandas/web_Scrapping_02.ipynb"

Consoles Page Succesfully Scrapped!
INTERMEDIATE.Dim_Consoles_Information Tables Successfully truncated! :D
Source File rows: 83
Inserted rows: 83
DB Insertion Finished in 0.0002362966537475586 minutes ! :D


### Needed variables

In [4]:
user_name = 'XXXXX' ## We won't save credentials on github! 
password =  'XXXXX' ## We won't save credentials on github!
dw_server = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
db_name = 'VideoGames_Sales'

SRC_Tables = ['Developer', 'Publisher', 'VideoGames']

connection_string = (
    f'mssql+pyodbc://{user_name}:{password}@{dw_server}/'
    f'{db_name}?driver=ODBC+Driver+17+for+SQL+Server'
)

# Connecting with SQL Server
engine = create_engine(connection_string, fast_executemany=True)

## Loading dimensions that go from SRC to DW without and INTERMEDIATE table

In [5]:
with engine.connect() as conn:
    for table_name in SRC_Tables:
        try:
            conn.execute(text(f"exec DW.SP_Load_Dim_{table_name}"))
            print(f"Dim_{table_name} Successfully loaded!")
            conn.commit()
        except Exception as e:
            raise ValueError(str(e))

Dim_Developer Successfully loaded!
Dim_Publisher Successfully loaded!
Dim_VideoGames Successfully loaded!


## Loading Dim_Consoles_Information, that goes from SRC -> INTERMEDIATE -> DW

In [6]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        conn.execute(text(f"exec DW.SP_Load_Intermediate_Dim_Consoles_Information"))
        print(f"INTERMEDIATE.Dim_Consoles_Information Successfully loaded!")

        ## INTERMEDIATE -> DW
        conn.execute(text(f"exec DW.SP_Load_Dim_Consoles_Information"))
        print(f"DW.Dim_Consoles_Information Successfully loaded!")

        conn.commit()
    except Exception as e:
        raise ValueError(str(e))

INTERMEDIATE.Dim_Consoles_Information Successfully loaded!
DW.Dim_Consoles_Information Successfully loaded!


## As a final step, we will load our Fact_VideoGames_Sales, that goes from SRC -> INTERMEDIATE -> DW

In [7]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        conn.execute(text(f"exec DW.SP_Load_Intermediate_Fact_VideoGames_Sales"))
        print(f"INTERMEDIATE.Fact_VideoGames_Sales Successfully loaded!")

        ## INTERMEDIATE -> DW
        conn.execute(text(f"exec DW.SP_Load_Fact_VideoGames_Sales"))
        print(f"DW.Fact_VideoGames_Sales Successfully loaded!")

        conn.commit()
    except Exception as e:
        raise ValueError(str(e))

INTERMEDIATE.Fact_VideoGames_Sales Successfully loaded!
DW.Fact_VideoGames_Sales Successfully loaded!


<h1 align = 'center'>Showing Dimensions, Facts and results</h1>
<h2 align = 'center'>(Please, check scripts to realize how are we loading them)</h2>
<h3 align = 'center'>¡¡¡TIMEZONE: ARGENTINA!!!</h3>

## Dimensions
Every Dimension have an unique key defined, so, We are 100% that It won't insert duplicated values. Otherwise, It would Fail
It also ensures that we won't have duplicated records when joining data from fact to our dimensions

### Dim_VideoGames

In [8]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text("select * from DW.Dim_VideoGames")).fetchall()
        Dim_VideoGames = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

Dim_VideoGames.sample(10)

,ID_VideoGame,Game_Name,Audit_Insert_TS,Audit_Insert_Username
5618,5618,STEAMWORLD QUEST: HAND OF GILGAMECH,2024-01-11 19:49:48.570,ETL System
14695,14695,CHOU MAJIN EIYUUDEN WATARU: MAZEKKO MONSTER,2024-01-11 19:49:48.570,ETL System
29679,29679,PINBALL MANIA,2024-01-11 19:49:48.570,ETL System
7724,7724,CHOU SENTOUCHUU: KYUUKYOKU NO SHINOBU TO BATTL...,2024-01-11 19:49:48.570,ETL System
25100,25100,M&M'S BREAK' EM,2024-01-11 19:49:48.570,ETL System
8580,8580,DISHONORED: DEFINITIVE EDITION,2024-01-11 19:49:48.570,ETL System
2899,2899,RISE OF THE TOMB RAIDER,2024-01-11 19:49:48.570,ETL System
16016,16016,DAREDEMO KANTAN! WATANABE AKIRA NO TSUME SHOGI,2024-01-11 19:49:48.570,ETL System
28541,28541,OBESITY EPIDEMIC,2024-01-11 19:49:48.570,ETL System
16858,16858,DISNEY'S DONALD DUCK: GOIN' QUACKERS,2024-01-11 19:49:48.570,ETL System


### Dim_Developer

In [9]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text("select * from DW.Dim_Developer")).fetchall()
        Dim_Developer = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

Dim_Developer.sample(10)

,ID_Developer,Developer,Audit_Insert_TS,Audit_Insert_Username
4007,4007,PSYGNOSIS,2024-01-11 19:49:48.460,ETL System
6379,6379,JACOB AARONSON,2024-01-11 19:49:48.460,ETL System
7900,7900,F. SCHNEIDER AND SUCCESS,2024-01-11 19:49:48.460,ETL System
6441,6441,HAKUDAKUKEI,2024-01-11 19:49:48.460,ETL System
8101,8101,SEARS,2024-01-11 19:49:48.460,ETL System
7368,7368,KAIKOMASTER,2024-01-11 19:49:48.460,ETL System
1562,1562,VERTEX POP,2024-01-11 19:49:48.460,ETL System
7171,7171,CONNOR ULLMANN,2024-01-11 19:49:48.460,ETL System
6247,6247,VECTOR UNIT STUDIOS,2024-01-11 19:49:48.460,ETL System
6479,6479,KAZE,2024-01-11 19:49:48.460,ETL System


### Dim_Publisher

In [10]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text("select * from DW.Dim_Publisher")).fetchall()
        Dim_Publisher = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

Dim_Publisher.sample(10)

,ID_Publisher,Publisher,Audit_Insert_TS,Audit_Insert_Username
2293,2293,TERA BOX,2024-01-11 19:49:48.506666,ETL System
1798,1798,SHINGAKUSHA,2024-01-11 19:49:48.506666,ETL System
796,796,LIKE CHARLIE,2024-01-11 19:49:48.506666,ETL System
715,715,DRAMATIC CREATE,2024-01-11 19:49:48.506666,ETL System
1042,1042,EVOLVED GAMES,2024-01-11 19:49:48.506666,ETL System
530,530,DIGITAL CYBERCHERRIES,2024-01-11 19:49:48.506666,ETL System
1652,1652,TDK CORE,2024-01-11 19:49:48.506666,ETL System
2495,2495,ASPECT,2024-01-11 19:49:48.506666,ETL System
2780,2780,BERKELEY,2024-01-11 19:49:48.506666,ETL System
1898,1898,"MERGE INTERACTIVE, INCORPORATED",2024-01-11 19:49:48.506666,ETL System


### Dim_Consoles_Information

In [11]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text("select * from DW.Dim_Consoles_Information")).fetchall()
        Dim_Consoles_Information = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

Dim_Consoles_Information.sample(10)

,ID_Console,Console_Abbreviation,Console_Name,Developer,Release_Date,Origin_Country,Generation,Type,Media_Type,Graphics,Online_Play,Predecessor,Successor,Audit_Insert_TS,Audit_Insert_Username,Audit_Update_TS,Audit_Update_Username,INCOMPLETE_INFORMATION
52,52,NGCD,NEO GEO CD,SNK,1994-09-09,JAPAN,FIFTH,HOME CONSOLE,CD-ROM,24-BIT,YES,NEO GEO,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,None,None,NO
116,116,BBCM,NOT SPECIFIED,NOT SPECIFIED,9999-01-01,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,None,None,YES
65,65,CD32,AMIGA CD32,COMMODORE,1993-09-17,USA,FIFTH,HOME CONSOLE,CD-ROM,32-BIT,YES,NOT SPECIFIED,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,None,None,NO
82,82,PVS,PALMTEX PORTABLE VIDEOGAME SYSTEM,PALMTEX,1982-01-01,USA,SECOND,HANDHELD,CARTRIDGE,8-BIT,NO,NOT SPECIFIED,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,None,None,NO
114,114,ZXS,NOT SPECIFIED,NOT SPECIFIED,9999-01-01,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,None,None,YES
11,11,GBA,GAME BOY ADVANCE,NINTENDO,2001-03-21,JAPAN,SIXTH,HANDHELD,PHYSICAL,2D/3D,NO,GB,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,None,None,NO
23,23,MS,SEGA MASTER SYSTEM,SEGA,1985-10-20,JAPAN,THIRD,HOME CONSOLE,CARTRIDGE,8-BIT,NO,SG,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,None,None,NO
99,99,PSN,NOT SPECIFIED,NOT SPECIFIED,9999-01-01,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,None,None,YES
36,36,SCD,SEGA CD,SEGA,1991-12-12,JAPAN,FIFTH,PERIPHERAL,CD-ROM,16-BIT,YES,GENESIS,DREAMCAST,2024-01-11 19:49:49.730,ETL System,None,None,NO
17,17,GEN,SEGA GENESIS,SEGA,1988-10-29,USA,FOURTH,HOME CONSOLE,CARTRIDGE,16-BIT,NO,SMS,NOT SPECIFIED,2024-01-11 19:49:49.730,ETL System,None,None,NO


!!!<br/>
Note that we have some values with INCOMPLETE_INFORMATION: NO. 
This means that those values were inserted just with their abbreviations.
In a future, we could update their information, but It will work for now.<br/>
!!!

## Fact Table
It has a primary key defined over its dimensions and Audit_Insert_TS field. As It is fixed by batch, It won't insert duplicated rows for the same day. Otherwise, It would fail.

In [12]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text("select * from DW.Fact_VideoGames_Sales")).fetchall()
        Fact_VideoGames_Sales = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

Fact_VideoGames_Sales.sample(10)

,ID_VideoGame,ID_Console,ID_Publisher,ID_Developer,VGChartz_Score,Critic_Score,User_Score,Total_Shipped,Total_Sales,NA_Sales,...,Japan_Sales,Other_Sales,ID_Release_Date,Last_Update,HASH_VALUE,Audit_Insert_TS,Audit_Insert_Username,Audit_Update_TS,Audit_Update_Username,IS_CURRENT_VERSION
56786,36075,99,0,806,None,None,None,NaN,NaN,NaN,...,NaN,NaN,20110208,None,b'\xf7^ <\x11_C\x0f&\x921\x8e\xc2+\x87dM\t\x92...,2024-01-11 19:49:49.570,ETL System,None,None,CURRENT VERSION
1612,569,16,0,327,None,None,None,NaN,NaN,NaN,...,NaN,NaN,0,2023-03-30,b'\x9a\x04]\xa7\xf1\xa2R=^*\xb3\xab\xa9e-\xd5\...,2024-01-11 19:49:49.570,ETL System,None,None,CURRENT VERSION
32319,18062,99,586,3822,None,None,None,NaN,NaN,NaN,...,NaN,NaN,20100728,None,b'(\x06\xc2`qkoP0\xf8\xad[H>\xf8e\xff\xea\x8d\...,2024-01-11 19:49:49.570,ETL System,None,None,CURRENT VERSION
50166,31206,105,1716,32,None,None,None,NaN,NaN,NaN,...,NaN,NaN,20100215,None,b'z\x89\x86\x9a\x00\xeb\xe8\xcb\x15\x98\xe5\xd...,2024-01-11 19:49:49.570,ETL System,None,None,CURRENT VERSION
8205,2936,91,9,30,None,None,None,9000000.0,NaN,NaN,...,NaN,NaN,20100302,2020-10-23,b'\xd5\x13\xcd\x1d\n^\xdag\xdc\xe0.\xfc\xa1w\x...,2024-01-11 19:49:49.570,ETL System,None,None,CURRENT VERSION
22972,11336,2,714,3860,None,None,None,NaN,NaN,NaN,...,NaN,NaN,20091217,None,"b'\xa2\xd7\xd5\xbc\xbd\x84w\xea\x92\x15J\xef,\...",2024-01-11 19:49:49.570,ETL System,None,None,CURRENT VERSION
18257,8295,14,16,792,None,None,None,NaN,1420000.0,770000.0,...,10000.0,120000.0,20140930,2018-05-04,b'\x88.\x9b\xce\xc6\x00\xd3\xea\x82}\x96\xbb\x...,2024-01-11 19:49:49.570,ETL System,None,None,CURRENT VERSION
38887,23001,2,1683,4216,None,None,None,NaN,0.0,NaN,...,NaN,0.0,20090421,None,b']\xa7\x9c\x02\xde\x08\xc5\x80\xa7\xbc3N\xf4\...,2024-01-11 19:49:49.570,ETL System,None,None,CURRENT VERSION
44273,26983,6,105,615,None,None,None,NaN,210000.0,120000.0,...,NaN,10000.0,20000630,None,b'\x12\x85\xdb\xb7\x0e:h\xd7\xed_\xbc\xe1\x90\...,2024-01-11 19:49:49.570,ETL System,None,None,CURRENT VERSION
20416,9509,19,1600,2626,None,None,None,NaN,NaN,NaN,...,NaN,NaN,19820101,None,"b""\rnCo\x18Y'\xee\x86\xaaJ\xc3\xdc`\xc3\xe2gw\...",2024-01-11 19:49:49.570,ETL System,None,None,CURRENT VERSION


### Full Data Report Sample

In [13]:
with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text(
        """
        select
		 VIDEOGAMES.Game_Name
		,CONSOLES.Console_Abbreviation
		,CONSOLES.Console_Name
		,CONSOLES.Generation AS Concole_Generation
		,Publisher.Publisher
		,Developer.Developer
		,VGChartz_Score
		,Critic_Score
		,User_Score
		,Total_Shipped
		,Total_Sales
		,NA_Sales
		,PAL_Sales
		,Japan_Sales
		,Other_Sales
		,DDATE.CalendarDate AS Release_Date
		,DDATE.CalendarYear AS Release_Date_Year
		,DDATE.CalendarMonthName AS Release_Date_Month
		,DDATE.FiscalDayOfMonth Release_Date_Fiscal_Day_Of_Month
	from 
		DW.Fact_VideoGames_Sales FACT
		left join DW.Dim_VideoGames VIDEOGAMES ON VIDEOGAMES.ID_VideoGame = FACT.ID_VideoGame
		left join DW.Dim_Consoles_Information CONSOLES ON CONSOLES.ID_Console = FACT.ID_Console
		left join DW.Dim_Publisher PUBLISHER ON PUBLISHER.ID_Publisher = FACT.ID_Publisher
		left join DW.Dim_Developer DEVELOPER ON DEVELOPER.ID_Developer = FACT.ID_Developer
		left join DW.Dim_Date DDATE ON DDATE.DateKey = FACT.ID_Release_Date
        """)).fetchall()
        df = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

df.sample(10)

,Game_Name,Console_Abbreviation,Console_Name,Concole_Generation,Publisher,Developer,VGChartz_Score,Critic_Score,User_Score,Total_Shipped,Total_Sales,NA_Sales,PAL_Sales,Japan_Sales,Other_Sales,Release_Date,Release_Date_Year,Release_Date_Month,Release_Date_Fiscal_Day_Of_Month
51699,HOTLINE MIAMI 2: WRONG NUMBER,PS4,PLAYSTATION 4,EIGHTH,DEVOLVER DIGITAL,NOT SPECIFIED,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,2015-03-10,2015,March,10
40978,MEDAROT 5: SUSUTAKE MURA NO TENKOUSEI: KABUTO ...,GB,GAME BOY,FOURTH,IMAGINEER,NATSUME,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,2001-12-14,2001,December,14
10725,INSIDE / LIMBO DOUBLE PACK,PS4,PLAYSTATION 4,EIGHTH,505 GAMES,PLAYDEAD,None,None,None,NaN,40000.0,40000.0,NaN,NaN,10000.0,2017-09-12,2017,September,12
44327,MEGA WORDS,DSIW,NOT SPECIFIED,NOT SPECIFIED,DIGITAL LEISURE INC.,DIGITAL LEISURE INC.,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,2010-06-21,2010,June,21
44618,OSMOS,PC,NOT SPECIFIED,NOT SPECIFIED,HEMISPHERE GAMES,HEMISPHERE GAMES,None,7.8,None,410000.0,NaN,NaN,NaN,NaN,NaN,2009-08-08,2009,August,8
15650,ARKEDO SERIES - 03 PIXEL!,XBL,NOT SPECIFIED,NOT SPECIFIED,MICROSOFT,ARKEDO,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,2009-12-21,2009,December,21
46115,EIKEN DS TRAINING,DS,NINTENDO DS,SEVENTH,IE INSTITUTE,IE INSTITUTE,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,2007-09-13,2007,September,13
61153,TOUGHMAN CONTEST,GEN,SEGA GENESIS,FOURTH,EA SPORTS,VISUAL CONCEPTS,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,1995-01-01,1995,January,1
60104,SUIKO ENBU: FUUUN SAIKI,SAT,SEGA SATURN,FIFTH,DATA EAST,DATA EAST,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,1996-03-22,1996,March,22
14507,SHREK: TREASURE HUNT,PS,PLAYSTATION,FIFTH,TDK MEDIACTIVE,THE CODE MONKEYS,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,2002-10-18,2002,October,18


### Showing Today's inserted/updated rows by entity

In [14]:
query = """
DECLARE @TODAY DATE = CAST(GETDATE() AS DATE)

SELECT
	'Dim_VideoGames' as Entity,
	COUNT(1) Total_Inserted_Rows,
	'Entity does not update data' as Total_Updated_Rows
FROM
	DW.Dim_VideoGames
WHERE
	CAST(Audit_Insert_TS AS DATE) = @TODAY

UNION ALL

SELECT
	'Dim_Developer' as Entity,
	COUNT(1) Total_Inserted_Rows,
	'Entity does not update data' as Total_Updated_Rows
FROM
	DW.Dim_Developer
WHERE
	CAST(Audit_Insert_TS AS DATE) = @TODAY

UNION ALL

SELECT
	'Dim_Publisher' as Entity,
	COUNT(1) Total_Inserted_Rows,
	'Entity does not update data' as Total_Updated_Rows
FROM
	DW.Dim_Publisher
WHERE
	CAST(Audit_Insert_TS AS DATE) = @TODAY

UNION ALL

SELECT
	'Dim_Consoles_Information' as Entity,
	SUM(CASE WHEN Audit_Update_TS IS NULL THEN 1 ELSE 0 END) Total_Inserted_Rows,
	CAST(SUM(CASE WHEN Audit_Update_TS IS NOT NULL THEN 1 ELSE 0 END) AS nvarchar(20)) as Total_Updated_Rows
FROM
	DW.Dim_Consoles_Information
WHERE
	CAST(Audit_Insert_TS AS DATE) = @TODAY OR CAST(Audit_Update_TS AS DATE) = @TODAY

UNION ALL

SELECT
	'Fact_VideoGames_Sales' as Entity,
	SUM(CASE WHEN Audit_Update_TS IS NULL THEN 1 ELSE 0 END) Total_Inserted_Rows,
	CAST(SUM(CASE WHEN Audit_Update_TS IS NOT NULL THEN 1 ELSE 0 END) AS nvarchar(20)) as Total_Updated_Rows
FROM
	DW.Fact_VideoGames_Sales
WHERE
	CAST(Audit_Insert_TS AS DATE) = @TODAY OR CAST(Audit_Update_TS AS DATE) = @TODAY
"""

with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text(query)).fetchall()
        df = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

df.sort_values(by=['Entity'], ascending=False)

,Entity,Total_Inserted_Rows,Total_Updated_Rows
4,Fact_VideoGames_Sales,63818,0
0,Dim_VideoGames,41056,Entity does not update data
2,Dim_Publisher,3382,Entity does not update data
1,Dim_Developer,8848,Entity does not update data
3,Dim_Consoles_Information,117,0


### Showing VideoGames History on Fact (We might not have any record with history yet)

In [15]:
query = """
with CTE_Changed_Records --We will use It as a filter
as
(
SELECT 
	ID_VideoGame,
	ID_Console,
	ID_Publisher,
	ID_Developer,
	count(1) NUMBER_OF_VERSIONS
FROM
	DW.Fact_VideoGames_Sales
GROUP BY
	ID_VideoGame,
	ID_Console,
	ID_Publisher,
	ID_Developer
HAVING
	COUNT(1) > 1
)

        select
		 VIDEOGAMES.Game_Name
		,CONSOLES.Console_Abbreviation
		,CONSOLES.Console_Name
		,CONSOLES.Generation AS Concole_Generation
		,Publisher.Publisher
		,Developer.Developer
		,VGChartz_Score
		,Critic_Score
		,User_Score
		,Total_Shipped
		,Total_Sales
		,NA_Sales
		,PAL_Sales
		,Japan_Sales
		,Other_Sales
		,DDATE.CalendarDate AS Release_Date
		,DDATE.CalendarYear AS Release_Date_Year
		,DDATE.CalendarMonthName AS Release_Date_Month
		,DDATE.FiscalDayOfMonth Release_Date_Fiscal_Day_Of_Month
		,Last_Update 
		,FACT.Audit_Insert_TS
		,FACT.Audit_Update_TS
		,FACT.IS_CURRENT_VERSION
	from 
		DW.Fact_VideoGames_Sales FACT
		INNER JOIN CTE_Changed_Records CR ON CR.ID_Console = FACT.ID_Console AND CR.ID_Developer = FACT.ID_Developer AND CR.ID_Publisher = FACT.ID_Publisher AND CR.ID_VideoGame = FACT.ID_VideoGame
		left join DW.Dim_VideoGames VIDEOGAMES ON VIDEOGAMES.ID_VideoGame = FACT.ID_VideoGame
		left join DW.Dim_Consoles_Information CONSOLES ON CONSOLES.ID_Console = FACT.ID_Console
		left join DW.Dim_Publisher PUBLISHER ON PUBLISHER.ID_Publisher = FACT.ID_Publisher
		left join DW.Dim_Developer DEVELOPER ON DEVELOPER.ID_Developer = FACT.ID_Developer
		left join DW.Dim_Date DDATE ON DDATE.DateKey = FACT.ID_Release_Date
	order by 1, 2, 3, 4, FACT.Audit_Insert_TS DESC
"""

with engine.connect() as conn:
    try:
        ## SRC -> INTERMEDIATE
        results = conn.execute(text(query)).fetchall()
        df = pd.DataFrame(results)
    except Exception as e:
        raise ValueError(str(e))

df

""


Closing connection properly

In [16]:
engine.dispose()

<h1 align='center'>Thanks For Watching! :D</h1>